In [2]:
from appgeopy import *
from my_packages import *

# ====================================================
# Configuration Parameters
# ====================================================
BUFFER_RADIUS = 100  # Buffer radius for InSAR point search, in meters
START_YEAR = 2016  # Start year for analysis
END_YEAR = 2023  # End year for analysis

# ====================================================
# File Paths
# ====================================================
LEVELING_SHP_PATH = r"E:\SUBSIDENCE_PROJECT_DATA\地陷資料整理\水準點\LevelingBenchmarks_DavidNCU\shapefiles\leveling_benchmark_location.shp"
VERTICAL_DISP_FILE_PATTERN = f"*{BUFFER_RADIUS}*.shp"
HDF5_FILE_PATH = r"D:\1000_SCRIPTS\003_Project002\20241219_LevelingData_to_HDF5\20241220_All_LevelingData_LandSubsidence-wra-gov-tw.h5"


VERTICAL_DISP_FOLDER = r"E:\001_InSAR_Project\1100_CHOUSHUI_2025\CRFP_full_2016_2024\asc_desc_run003L"

# for VERTICAL_DISP_FOLDER in tqdm(
#     glob(os.path.join(r"E:\001_InSAR_Project\1100_CHOUSHUI_2025\CRFP_full_2016_2024", "asc_desc_run*"))
# ):

print(f"Current folder: {VERTICAL_DISP_FOLDER}")
# ====================================================
# Load Data
# ====================================================
print("Loading leveling benchmark shapefile...")
leveling_shp = gpd.read_file(LEVELING_SHP_PATH)

print("Searching for vertical displacement shapefile...")
vertical_disp_fpath = glob(os.path.join(VERTICAL_DISP_FOLDER, VERTICAL_DISP_FILE_PATTERN))
if not vertical_disp_fpath:
    raise FileNotFoundError("No matching vertical displacement shapefile found.")
vertical_disp_fpath = vertical_disp_fpath[0]
vertical_disp_shp = gpd.read_file(vertical_disp_fpath)

# ====================================================
# Find Neighbors for Each Benchmark
# ====================================================
CENTRAL_KEY_COLUMN = "樁號"  # Key column for benchmark identifier
results = []

print("Finding neighbors for each leveling benchmark...")
for idx, row in tqdm(leveling_shp.iterrows(), total=len(leveling_shp)):
    try:
        result = geospatial.find_point_neighbors(row, vertical_disp_shp, CENTRAL_KEY_COLUMN, BUFFER_RADIUS)
        results.append(result)
    except Exception as e:
        print(f"Error processing row {idx}: {e}")

# Combine results into a single GeoDataFrame
result_gdf = pd.concat(results, ignore_index=True)

# Save results as a pickle file
save_path = os.path.join(
    VERTICAL_DISP_FOLDER, os.path.basename(vertical_disp_fpath).replace(".shp", "_add_StationNum.pkl")
)
result_gdf.to_pickle(save_path)

# ====================================================
# Load HDF5 Leveling Data
# ====================================================
print("Loading leveling data from HDF5 file...")
with h5py.File(HDF5_FILE_PATH, "r") as hdf5_file:
    leveling_measure_dict = gwatertools.h5pytools.hdf5_to_data_dict(hdf5_file)
    leveling_metadata_dict = gwatertools.h5pytools.hdf5_to_metadata_dict(hdf5_file)

# Extract unique benchmarks in the buffer zone
unique_leveling_StationNumber = result_gdf[CENTRAL_KEY_COLUMN].unique()

# Filter valid benchmarks available in the database
valid_leveling_StationNumber = sorted(
    set(leveling_metadata_dict.keys()).intersection(set(unique_leveling_StationNumber))
)

# ====================================================
# Filter Benchmarks by Time Interval
# ====================================================
satisfied_leveling_StationNumber = []
print("Filtering benchmarks by time interval...")
for select_StationNumber in tqdm(valid_leveling_StationNumber):
    metadata = leveling_metadata_dict[select_StationNumber]
    first_record_date = pd.to_datetime(metadata["First_Record"])
    last_record_date = pd.to_datetime(metadata["Last_Record"])
    if first_record_date.year <= START_YEAR and last_record_date.year >= END_YEAR:
        satisfied_leveling_StationNumber.append(select_StationNumber)

# ====================================================
# Compare Leveling and InSAR Measurements
# ====================================================
cache = {"StationNumber": [], "Leveling_cm_yr": [], "InSAR_cm_yr": []}

print("Comparing leveling and InSAR measurements...")
for select_StationNumber in tqdm(satisfied_leveling_StationNumber):
    try:
        # Leveling Measurements
        leveling_data = pd.DataFrame(leveling_measure_dict[select_StationNumber])
        leveling_data["date"] = leveling_data["date"].apply(lambda x: x.decode("utf-8"))
        leveling_data["date"] = pd.to_datetime(leveling_data["date"])
        leveling_data.set_index("date", inplace=True)
        subset_leveling_data = leveling_data.loc[str(START_YEAR) : str(END_YEAR)]

        # Calculate leveling velocity
        index_arr = subset_leveling_data.index
        y_arr = subset_leveling_data["values"].to_numpy()
        days_diff = np.array((index_arr - index_arr[0]).days)
        trend, coeffs = analysis.get_polynomial_trend(days_diff, y_arr, order=1)
        leveling_slope = coeffs[-1] * 365.25  # m/year
        leveling_slope_cm = leveling_slope * 100

        # InSAR Measurements
        insar_data = result_gdf.query(f"{CENTRAL_KEY_COLUMN}==@select_StationNumber")
        insar_velocity = insar_data["grid_code"].mean() * 100  # Convert to cm/year

        # Append results to cache
        cache["StationNumber"].append(select_StationNumber)
        cache["Leveling_cm_yr"].append(leveling_slope_cm)
        cache["InSAR_cm_yr"].append(insar_velocity)
    except Exception as e:
        print(f"Error processing {select_StationNumber}: {e}")

# ====================================================
# Save Results
# ====================================================
output_path = os.path.join(os.path.dirname(save_path), f"Leveling_InSAR_inBuffer_{BUFFER_RADIUS}m.xlsx")
leveling_insar_df = pd.DataFrame(cache)
leveling_insar_df.to_excel(output_path, index=False)

print(f"Results saved to {output_path}")
print()

Current folder: E:\001_InSAR_Project\1100_CHOUSHUI_2025\CRFP_full_2016_2024\asc_desc_run003K
Loading leveling benchmark shapefile...
Searching for vertical displacement shapefile...
Finding neighbors for each leveling benchmark...


  0%|          | 0/1858 [00:00<?, ?it/s]

Loading leveling data from HDF5 file...
Filtering benchmarks by time interval...


  0%|          | 0/1228 [00:00<?, ?it/s]

Comparing leveling and InSAR measurements...


  0%|          | 0/664 [00:00<?, ?it/s]

Results saved to E:\001_InSAR_Project\1100_CHOUSHUI_2025\CRFP_full_2016_2024\asc_desc_run003K\Leveling_InSAR_inBuffer_100m.xlsx

